In [1]:
    # MTG 1/6
    #POStagとENTtagの間には何らかの関係性が存在？
    #NERにおけるPOStagアプローチはsimpleと何も変わらないのでは＝＞誤差レベルの違いである可能性が高い
    #構文木のほうにいけるとよいのですが...

In [28]:
import torch
device = torch.device("cuda:1")

# data 拡張

## train

In [1]:
with open('data_original/train.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    data = data[1:14987] #不要部分をカット

In [2]:
# storing conll format data in multiple list
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split(" ")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)
    
    # [[['-DOCSTART-', '-X-', '-X-', 'O']],
    #  [['EU', 'NNP', 'B-NP', 'B-ORG'],
    #   ['rejects', 'VBZ', 'B-VP', 'O'],
    #   ['German', 'JJ', 'B-NP', 'B-MISC'],...

In [3]:
# ['B-ADVP', 'I-ADVP', 'I-ADJP', 'I-SBAR',
#  'I-PP', 'B-LST', 'B-SBAR', 'B-NP',
#  'B-ADJP', 'I-INTJ', 'B-VP', '-X-',
#  'I-CONJP', 'B-PRT', 'B-CONJP', 'I-LST',
#  'B-INTJ', 'B-PP', 'I-VP', 'O', 'I-NP']

# I-B I-other cut
#    I-I stay 
# B-I
#   B-B B-oter cut

In [40]:
new_document = []

for sentence in document:
    sentence_list = []
    
    for i in range(len(sentence)-1):
        sentence_list.append(sentence[i])
        
        if "I-" in sentence[i][2]:
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
                
        else: #B,other
            if "I-" in sentence[i+1][2]:
                pass
            else: #B,other
                sentence_list.append(-1)
    
    final_sentence_list = []
    new_tokens_list = []
    all_token = []
    syntactic_tag = ""
    bio_tag = []
    
    for token in sentence_list:
        if token == -1 or token==sentence_list[-1]:
            
            new_tokens_list.append(tuple(all_token))
            new_tokens_list.append(syntactic_tag)#タグ生成
            new_tokens_list.append(tuple(bio_tag))
            
            final_sentence_list.append(new_tokens_list)
            
            new_tokens_list = []
            all_token = []
            syntactic_tag = ""
            bio_tag = []
        else:
            all_token.append(token[0])
            if syntactic_tag == "":
                syntactic_tag=token[2]#タグ生成
            bio_tag.append(token[3])
            
            
    new_document.append(final_sentence_list)
    
    
# new_document = []
# for sentence in document:
#     sentence_list = []
    
#     for i in range(len(sentence)-1):
#         sentence_list.append(sentence[i])
        
#         if "I-" in sentence[i][2]:
#             if "I-" in sentence[i+1][2]:
#                 pass
#             else: #B,other
#                 sentence_list.append(-1)
                
#         else: #B,other
#             if "I-" in sentence[i+1][2]:
#                 pass
#             else: #B,other
#                 sentence_list.append(-1)
    
#     final_sentence_list = []
#     new_tokens_list = []
#     syntactic_tag = set()
#     bio_tag = set()
    
#     for token in sentence_list:
#         if token == -1 or token==sentence_list[-1]:
            
#             new_tokens_list.append(syntactic_tag)#タグ生成
#             new_tokens_list.append(bio_tag)
            
#             final_sentence_list.append(new_tokens_list)
#             new_tokens_list = []
            
#             syntactic_tag = set()
#             bio_tag = set()
#         else:
#             syntactic_tag.add(token[2])#タグ生成
#             bio_tag.add(token[3])
            
#             new_tokens_list.append([token[0],token[2],token[3]])
        
#     new_document.append(final_sentence_list)
    

In [47]:
new_document[0:2]

[[[('EU',), 'B-NP', ('B-ORG',)],
  [('rejects',), 'B-VP', ('O',)],
  [('German', 'call'), 'B-NP', ('B-MISC', 'O')],
  [('to', 'boycott'), 'B-VP', ('O', 'O')],
  [('British', 'lamb'), 'B-NP', ('B-MISC', 'O')]],
 [[(), '', ()]]]

In [52]:
# とりあえず，一ブロックに一単語のみ格納されているケースは削除
# 2以上のケース　かつ　ブロック内のBOIタグ構造が完全に一致するものだけを交換

In [60]:
# making dictonary object with label as key and token as value
label_token_dic = {}
cnt = 0
for sentence in new_document:
    for block in sentence:
        if len(block[0])!=1:# block内にトークン2以上の時のみ辞書登録
            label = tuple(block[-2:])
            cnt = cnt + 1
            if label not in label_token_dic:
                label_token_dic[label] = set()
            label_token_dic[label].add(tuple(block[0]))
        
    # {('B-NP', ('B-MISC', 'O')): {('A$', '1'),
    #   ('A$', '100'),
    #   ('A$', '14,000'),
    #   ('A$', '28,000'),...
    
    # label_token_dic.keys() でkey一覧取得可　個数：663
    # cnt　に合計要素数格納　個数：41377

In [84]:
# making BI label list
# BIを含むもののみ交換

BI_label_list = []
for label in list(label_token_dic.keys()):
    NG_set = [{"O"},set(),{"-X-"},{"O","-X-"}]
    if set(label[1]) not in NG_set:
        BI_label_list.append(label)
        
# [('B-NP', ('B-MISC', 'O')),
#  ('B-NP', ('O', 'B-ORG', 'I-ORG')),
#  ('B-NP', ('O', 'O', 'O', 'B-PER', 'I-PER')),...

In [86]:
def Sent2str(sentence_list):
    return " ".join([token_list[0] for token_list in sentence_list])

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

In [114]:
import random
import csv


def SyntacticDA_BI(n,filename):
    data_f_path = "data_augment/SyntacticDA_BI/"+str(n)+"_"+filename+".txt"
    memo_f_path = "data_augment/SyntacticDA_BI/"+str(n)+"_"+filename+"_memo.txt"
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for now in range(n):
            if now%10 == 0:
                print('\r%d / %d' %(now, n), end='')

            x = random.randint(0,len(new_document)-1)
            origin_sentence = new_document[x]
            aug_sentence = []
            cnt = 0

            for token_list in origin_sentence:
                token = token_list[0]
                label = token_list[1]
                origin_label = token_list[2]

                if label in BI_label_list:
                    if len(label_token_dic[label])!=1:
                        token=random_choice_token(label_token_dic[label]-{token})
                        cnt = cnt + 1
                    

                aug_sentence.append([token,origin_label])
                data_f.write(" ".join([token,origin_label])+"\n")
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+Sent2str(new_document[x])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+Sent2str(aug_sentence)+"\n")
            memo_f.write("\n")
        
        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [115]:
# #train
n = 15000

In [116]:
SimpleDA_BI(n*1,"train")
SimpleDA_BI(n*4,"train")
SimpleDA_BI(n*9,"train")
SimpleDA_BI(n*49,"train")
SimpleDA_BI(n*99,"train")

1484990 / 1485000

## valid 

In [117]:
with open('data_original/valid.txt', 'r', encoding='UTF-8') as f:
    data = f.read()
    data = data.split("\n\n")
    data = data[1:3466]

In [118]:
# 以下同様
document = []
for sentence in data:#14988 sentences
    
    new_sentence_list = []
    sentence_list = sentence.split("\n")
    
    for token in sentence_list:
        token_list = token.split(" ")
        new_sentence_list.append(token_list)
    document.append(new_sentence_list)

new_document = []
for sentence in document:
    new_sentence_list = []
    for token_list in sentence:
        try:
            new_token_list = [token_list[0],token_list[1]+"_"+token_list[3],token_list[3]]
            new_sentence_list.append(new_token_list)
        except:
            pass
    new_document.append(new_sentence_list)

label_token_dic = {}
for sentence in new_document:
    for token_list in sentence:
        label = token_list[1]
        token = token_list[0]
        if label not in label_token_dic:
            label_token_dic[label] = set()
        label_token_dic[label].add(token)

        BI_label_list = [label for label in list(label_token_dic.keys()) if "B-" in label or "I-" in label]

BI_label_list = [label for label in list(label_token_dic.keys()) if "B-" in label or "I-" in label]

In [119]:
#valid
n = 3500

In [120]:
SimpleDA_BI(n*1,"valid")
SimpleDA_BI(n*4,"valid")
SimpleDA_BI(n*9,"valid")
SimpleDA_BI(n*49,"valid")
SimpleDA_BI(n*99,"valid")

346490 / 346500